(basic_fe_wave)=
# A basic FE method for the acoustic wave equation

[download as jupyter notebook](https://markuswess.github.io/waves/_sources/first_numerics/wave_eq.ipynb)

We follow the [method of lines](mol) approach for Galerkin methods to discretize the second order wave equation {eq}`wave_2o` on a spacial domain $\Omega:=(0,1)\times(0,1)$.

In `NGSolve` the domain $(0,1)\times(0,1)$ is readily available as `unit_square`

In [ ]:
# import ngsolve and webgui
from ngsolve import *
from ngsolve.webgui import Draw

# draw the unit_square shape
Draw(unit_square.shape);

but can also be generated by hand as follows

In [ ]:
# import occ geometry tools
from netgen.occ import *

unit_square_wp = Rectangle(1,1) # this actually returns a WorkPlane object
Draw(unit_square_wp.Face());

 Specifically we pick a so-called finite-element method for the spacial discretization.
In our context this means that the domain $\Omega$ is split up into a family of disjoint open subdomains $\mathcal T$ such that
```{math}
\bar{\Omega} = \bigcup_{T\in\mathcal T}\bar T
```
The subdomains $T$ are usually triangles or quadrilaterals (in 2d) or tetrahedra or hexahedra (in 3d) but in theory can take any shape. The decomposition into subdomains is called *meshing* and the set $\mathcal T$ is called *mesh*. In `NGSolve` this can be done as follows where the parameter `maxh` defines the size of the triangles.

In [ ]:
geo = OCCGeometry(unit_square.shape, dim = 2) # explicitely state dimension, otherwise dx integrals will vanish
mesh = Mesh(geo.GenerateMesh(maxh = 0.05))
Draw(mesh);

Next we need to define basis functions for the discrete space $V$. In this example this is done using the command `H1` as follows.

In [ ]:
V = H1(mesh)

A function $u\in V$ is called `GridFunction` and is represented by a coefficient vector.

In [ ]:
gfu = GridFunction(V)

print(gfu.vec)

By setting single entries of `gfu.vec` to one we obtain the respective basis functions.

In [ ]:
scene = Draw (gfu)
from time import sleep
#sleep(5)

for i in range(V.ndof):
    gfu.vec[:] = 0
    gfu.vec[i] = 1
    scene.Redraw()
    #sleep(1)

We want to solve the semi-discrete weak problem to find $u_h\in C^2([0,T];V)$

```{math}
:label: weak_semidisc
\begin{aligned}
\int_\Omega \partial_t^2 u_h(t,x)u_h'(x)dx+\int_\Omega\nabla u_h(t,x)\cdot\nabla u'_h(x)dx&=0,\\
u_h(0,\cdot) &= u_{h,0}
\end{aligned}
```
for all $u_h'\in V$.
Or equivalently if $u_h,u_{h,0}$ are represented by the coefficient vectors $\mathbf u(t)$, $\mathbf u^0$

```{math}
:label: disc_ode
\begin{aligned}
\frac{d^2}{dt}\mathbf M \mathbf u(t)+\mathbf S\mathbf u(t) &= 0\\
\mathbf u(0) &= \mathbf u^0
\end{aligned}
```
where the matrices $\mathbf M,\mathbf S$ are defined by
```{math}
\begin{aligned}
(\mathbf M)_{i,j}&:=\int_\Omega b_j(x)b_i(x)dx,&
(\mathbf S)_{i,j}&:=\int_\Omega \nabla b_j(x)\cdot \nabla b_i(x)dx
\end{aligned}
```
for a basis $b_0,\ldots,b_N$ of $V$.

To this end we need to assemble the matrices $\mathbf S,\mathbf M$. In `NGSolve` this can be done by

In [ ]:
u,u_ = V.TnT() # get test and trial functions

M = BilinearForm(u*u_*dx).Assemble()
S = BilinearForm(grad(u)*grad(u_)*dx).Assemble()

print(M.mat)

Approximating the initial conditions (we choose $u_0=\exp(-10(x^2+y^2))$) can be done using `GridFunction.Set`

In [ ]:
gfu.Set(exp(-10*(x**2+y**2)))
Draw(gfu);

It remains to solve {eq}`disc_ode` using an appropriate time stepping.

We choose a variant of the Newmark time-stepping which is given for problems of the form
```{math}
A\frac{d^2}{dt} x + B\frac{d}{dt}x+Cx=r,\quad
x(0)=x_0,\quad \frac{d}{dt}x(0)=0
```
by the approximations $x^j\approx x(\tau j)$, $\dot x^j\approx \frac{d}{dt} x(\tau j)$ and a time step size $\tau>0$

```{math}
\begin{aligned}
x^0&=x_0\\
\dot x^0&=0\\
x^{j+1}&=x^j+\frac{\tau}{2}(\dot x^{j+1}+\dot x^{j})\\
A\dot x^{j+1}&=A\dot x^j+\frac{\tau}{2}\left(r^{j+1}+r^j-C(x^{j+1}+x^{j})-B(\dot x^{j+1}+\dot x^{j})\right),
\end{aligned}
```
where we also set $r^j=r(\tau j)$.
Eliminating $x^{j+1}$ on the right hand side of the last equation and collecting the coefficients of $\dot x^{j+1}$ leads to

```{math}
\left(A+\frac{\tau}{2}B+\frac{\tau^2}{4}C\right)\dot x^{j+1}=A\dot x^j+\frac{\tau}{2}\left(r^{j+1}+r^j-B\dot x^j-C(2x^j+\frac{\tau}{2}\dot x^j)\right)

```

Note that in our example $A=\mathbf M$, $C=\mathbf S$, $r=0$, $B=0$.

We precompute the inverse of the matrix $\mathbf M^*=M+\frac{\tau^2}{4}\mathbf S$

In [ ]:
tau = 0.01
mstarinv = BilinearForm(u*u_*dx+tau**2/4* grad(u)*grad(u_)*dx).Assemble().mat.Inverse()


We define the approximation to the time derivative as a vector (and not a `GridFunction`) only and start the time loop

In [ ]:
T = 1
udot = gfu.vec.CreateVector()
udot[:] = 0.

scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data -= tau * mstarinv*(S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()

## Boundary conditions

Since we assumed $\nabla u\cdot n=0$ we ommitted the boundary term from our weak formulation {eq}`weak_space`. If we impose boundary conditions $\nabla u_h(t,\cdot)\cdot n=g_h(t,\cdot)$ on (a part of) the boundary $\Gamma=\partial\Omega$ we obtain
```{math}
\int_{\Gamma}\nabla u_h(t,x)\cdot n(x) u_h'(x) dS(x)=
\int_{\Gamma}g_h(t,x) u_h'(x) dS(x)
``` 
and thus the semi-discrete weak formulation

```{math}
:label: weak_semidisc_bd
\begin{aligned}
\int_\Omega \partial_t^2 u_h(t,x)u_h'(x)dx+\int_\Omega\nabla u_h(t,x)\cdot\nabla u'_h(x)dx&=\int_{\Gamma}g_h(t,x) u_h'(x) dS(x),\\
u_h(0,\cdot) &= u_{h,0}
\end{aligned}
```
for all $u_h'\in V$.
Note that the boundary term is now independent of the unknown function $u_h$ and thus acts as a right-hand-side.

To implement this in NGSolve we first need to make sure that we can adress the part of the boundary $\Gamma$ correctly. We pick the left edge of the `unit_square`, which is conveniently already named correctly.

In [ ]:
print(mesh.GetBoundaries())

Boundaries can also be named manually, e.g.

In [ ]:
# import occ geometry tools
from netgen.occ import *

square = Rectangle(1,1).Face() # this actually returns a WorkPlane object
square.edges.Min(X).name = "left"

Choosing $g=cos(\omega t)\exp(-10(y-0.5)^2)$ for some $\omega>0$ we assemble the spacial part of the boundary term as a  `LinearForm` similar to the `BilinearForm`. The boundary integral can be evaluated using `ds` as follows:

In [ ]:
f = LinearForm(exp(-10*(y-0.5)**2)*u_*ds("left")).Assemble()

It remains to include the right-hand-side in the time-stepping:

In [ ]:
T = 1
udot = gfu.vec.CreateVector()
udot[:] = 0.
gfu.vec[:] = 0.
omega = 15

scene = Draw(gfu, deformation=True)
for j in range(int(T/tau)):
    gfu.vec.data += tau/2 * udot
    udot.data += tau * mstarinv*(1/2*(cos(j*tau*omega)+cos((j+1)*tau*omega))*f.vec-S.mat*gfu.vec)
    gfu.vec.data += tau/2 * udot
    scene.Redraw()